In [8]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
nx, ny = 128, 128  # Grid size
rho0 = 1.0  # Initial density
tau = 0.6  # Relaxation time
niters = 5000  # Number of iterations

# Lattice weights and velocities for D2Q9
w = [4/9] + [1/9] * 4 + [1/36] * 4
dx = [0, 1, 0, -1, 0, 1, -1, -1, 1]

# Function to initialize the density and velocity fields
def initialize():
    f1 = np.ones((nx, ny, 9)) * rho0 / 9.0
    u = np.zeros((nx, ny, 2))
    return f1, u

# Function to compute equilibrium distribution
def equilibrium(rho, u):
    cu = np.dot(dx, u.transpose(2, 0, 1)).transpose(1, 2, 0)
    usqr = 3/2 * (u**2).sum(axis=2)
    feq = np.zeros((nx, ny, 9))
    
    for k in range(9):
        cu_k = cu[:, :, k].reshape((nx, ny))
        feq[:, :, k] = rho * w[k] * (1 + 3*cu_k + 9/2*cu_k**2 - 3/2*usqr)
    
    return feq


# Function to perform a single LBM iteration
def lbm_iteration(f1, u):
    # Collision step
    rho = f1.sum(axis=2)
    feq = equilibrium(rho, u)
    f1 += 1/tau * (feq - f1)

    # Streaming step
    for k in range(9):
        f1[:, :, k] = np.roll(f1[:, :, k], dx[k], axis=(0, 1))

    # Boundary conditions: Lid-driven cavity
    u[:, -1, 0] = 0.1  # Constant velocity on the lid

    # Update macroscopic variables
    rho = f1.sum(axis=2)
    u[:, :, 0] = (np.dot(w, dx) * f1.transpose(2, 0, 1)).transpose(1, 2, 0) / rho
    u[:, :, 1] = (np.dot(w, dx[1:]) * f1.transpose(2, 0, 1)).transpose(1, 2, 0) / rho

    return f1, u

# Main simulation loop
f1, u = initialize()

for it in range(niters):
    f1, u = lbm_iteration(f1, u)

# Plot the final velocity field
plt.imshow(np.sqrt(u[:, :, 0]**2 + u[:, :, 1]**2).transpose(), cmap='viridis', origin='lower', extent=[0, nx, 0, ny])
plt.colorbar(label='Velocity Magnitude')
plt.title('Lid-Driven Cavity Flow')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()


ValueError: shapes (9,) and (2,128,128) not aligned: 9 (dim 0) != 128 (dim 1)